## Choose model

In [2]:
model_name = 'distilbert-base-cased'

## Load dataset

In [4]:
from datasets import load_dataset, DatasetDict

In [5]:
dataset = load_dataset("jonathanli/law-stack-exchange")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'LastActivityDate', 'AnswerCount', 'CommentCount', 'ContentLicense', 'body', 'text_label', 'title'],
        num_rows: 638
    })
    validation: Dataset({
        features: ['Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'LastActivityDate', 'AnswerCount', 'CommentCount', 'ContentLicense', 'body', 'text_label', 'title'],
        num_rows: 319
    })
    test: Dataset({
        features: ['Id', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'LastActivityDate', 'AnswerCount', 'CommentCount', 'ContentLicense', 'body', 'text_label', 'title'],
        num_rows: 1596
    })
})

In [7]:
dataset['train'][0]

{'Id': 71340,
 'PostTypeId': 1,
 'CreationDate': '2021-09-01T02:39:58.830',
 'Score': 0,
 'ViewCount': 128.0,
 'LastActivityDate': '2021-09-01T06:43:55.503',
 'AnswerCount': 1.0,
 'CommentCount': 9,
 'ContentLicense': 'CC BY-SA 4.0',
 'body': "I'm trying to comfortably use my laptop at night and now could not find an answer by web-search to the following: Why TVs / laptops etc. do not warn about too bright screen whereas many smartphones warn about too loud sound amplification? I'm asking on law site cause I suspect there were a number of successful lawsuits for impaired hearing from headphones but none for impaired vision from TVs. Am I right here? Digging down - has it happened because there are studies proving impairment for one and none for the other?\n\n\nAdded: I actually doubt my second guess cause there are a lot of enhancements in screen brightness - auto-brightness, night mode.\n\n\n",
 'text_label': 'civil-law',
 'title': 'Why TVs / laptops etc. do not warn about too brirght

## Prepare data

In [9]:
dataset = dataset.remove_columns(["Id", "PostTypeId", "CreationDate", "Score", "ViewCount", "LastActivityDate",
                                 "AnswerCount", "CommentCount", "ContentLicense", "title"])

In [10]:
dataset = dataset.rename_column("text_label", "label")
dataset = dataset.rename_column("body", "text")

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 638
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 319
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1596
    })
})

In [12]:
import pandas as pd

In [13]:
train_labels = pd.DataFrame(dataset['train']['label']).value_counts()
train_labels.plot(kind='barh')

<Axes: ylabel='0'>

In [14]:
# balance the dataset and take just 4 classes
from datasets import concatenate_datasets

NUM_TRAINING_SAMPLES = 30
NUM_VAL_SAMPLES = 7
NUM_TESTING_SAMPLES = 5

# criminal-law 
label_6_train_data = dataset['train'].filter(lambda example: example["label"] == 'criminal-law').select(range(NUM_TRAINING_SAMPLES))
label_6_val_data = dataset['validation'].filter(lambda example: example["label"] == 'criminal-law').select(range(NUM_VAL_SAMPLES))
label_6_test_data = dataset['test'].filter(lambda example: example["label"] == 'criminal-law').select(range(NUM_TESTING_SAMPLES))
# employment (7)
label_7_train_data = dataset['train'].filter(lambda example: example["label"] == 'employment').select(range(NUM_TRAINING_SAMPLES))
label_7_val_data = dataset['validation'].filter(lambda example: example["label"] == 'employment').select(range(NUM_VAL_SAMPLES))
label_7_test_data = dataset['test'].filter(lambda example: example["label"] == 'employment').select(range(NUM_TESTING_SAMPLES))
# tax-law (14)
label_14_train_data = dataset['train'].filter(lambda example: example["label"] == 'tax-law').select(range(NUM_TRAINING_SAMPLES))
label_14_val_data = dataset['validation'].filter(lambda example: example["label"] == 'tax-law').select(range(NUM_VAL_SAMPLES))
label_14_test_data = dataset['test'].filter(lambda example: example["label"] == 'tax-law').select(range(NUM_TESTING_SAMPLES))
# trademark (15)
label_15_train_data = dataset['train'].filter(lambda example: example["label"] == 'trademark').select(range(NUM_TRAINING_SAMPLES))
label_15_val_data = dataset['validation'].filter(lambda example: example["label"] == 'trademark').select(range(NUM_VAL_SAMPLES))
label_15_test_data = dataset['test'].filter(lambda example: example["label"] == 'trademark').select(range(NUM_TESTING_SAMPLES))

Filter:   0%|          | 0/319 [00:00<?, ? examples/s]

Filter:   0%|          | 0/319 [00:00<?, ? examples/s]

Filter:   0%|          | 0/319 [00:00<?, ? examples/s]

Filter:   0%|          | 0/319 [00:00<?, ? examples/s]

In [15]:
train_data = concatenate_datasets([label_6_train_data, label_7_train_data, label_14_train_data, label_15_train_data])
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 120
})

In [16]:
val_data = concatenate_datasets([label_6_val_data, label_7_val_data, label_14_val_data, label_15_val_data])
val_data

Dataset({
    features: ['text', 'label'],
    num_rows: 28
})

In [17]:
test_data = concatenate_datasets([label_6_test_data, label_7_test_data, label_14_test_data, label_15_test_data])
test_data

Dataset({
    features: ['text', 'label'],
    num_rows: 20
})

In [18]:
NUM_LABELS = 4

In [19]:
# need to properly encode the labels 
from datasets import Dataset 

In [20]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

In [21]:
def encode_labels(dataset):
    encoded_data = pd.DataFrame(dataset)
    encoded_data = Dataset.from_pandas(encoded_data)
    encoded_data = encoded_data.class_encode_column("label")
    return encoded_data

In [22]:
train_data = encode_labels(train_data)
val_data = encode_labels(val_data)
test_data = encode_labels(test_data)

Casting to class labels:   0%|          | 0/120 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/28 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/20 [00:00<?, ? examples/s]

In [23]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['criminal-law', 'employment', 'tax-law', 'trademark'], id=None)}

In [24]:
# create id2label and label2id
labels = train_data.features['label'].names
ids = range(NUM_LABELS)
id2label = dict(zip(ids, labels))   
label2id = dict(zip(labels, ids)) 

In [25]:
# tokenize
from transformers import AutoTokenizer

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [33]:
tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [35]:
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_val = tokenized_val.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

In [37]:
tokenized_train

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 120
})

## Train model

In [39]:
from transformers import AutoModelForSequenceClassification

In [41]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS, label2id=label2id, id2label=id2label)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
from transformers import TrainingArguments

In [45]:
training_args = TrainingArguments(output_dir="my_test_model", eval_strategy="epoch")

In [47]:
import numpy as np
import evaluate

In [49]:
metric = evaluate.load("accuracy")

In [50]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [53]:
from transformers import Trainer

In [55]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

In [57]:
trainer.train()

TrainOutput(global_step=45, training_loss=0.8562598334418403, metrics={'train_runtime': 515.0012, 'train_samples_per_second': 0.699, 'train_steps_per_second': 0.087, 'total_flos': 47689964421120.0, 'train_loss': 0.8562598334418403, 'epoch': 3.0})

## Evaluate model

In [61]:
trainer.evaluate(tokenized_test)

{'eval_loss': 0.4440459609031677,
 'eval_accuracy': 0.95,
 'eval_runtime': 6.4454,
 'eval_samples_per_second': 3.103,
 'eval_steps_per_second': 0.465,
 'epoch': 3.0}

## Hyperparameter tuning

In [46]:
import optuna

In [47]:
# initialise a new model for each tuning run
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS, label2id=label2id, id2label=id2label)

In [48]:
# function to control the search of hyperparams to tune
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2, 3, 5])
    }

In [49]:
# create the training args for each tuning run
tuning_training_args = TrainingArguments(
    output_dir="model_tuning_output",
    eval_strategy="epoch",
    learning_rate=0.1,  # Placeholder; will be overridden by the hyperparameter search
    per_device_train_batch_size=16,  # Placeholder; will be overridden by the hyperparameter search
    num_train_epochs=3,  # Placeholder; will be overridden by the hyperparameter search
    logging_dir="./logs",
    #push_to_hub=True
)

In [50]:
#from huggingface_hub import notebook_login
#notebook_login()

In [51]:
tuning_trainer = Trainer(
    model_init=model_init,
    args=tuning_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
best_trial = tuning_trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    n_trials=5
)

[I 2025-02-01 16:08:23,814] A new study created in memory with name: no-name-2be3205c-c33e-4b02-8b56-2ae2b7ccd655
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-02-01 16:13:34,507] Trial 0 finished with value: 0.9642857142857143 and parameters: {'learning_rate': 0.0002728032165901902, 'per_device_train_batch_size': 16, 'num_train_epochs': 2}. Best is trial 0 with value: 0.9642857142857143.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-02-01 16:18:50,407] Trial 1 finished with value: 0.8928571428571429 and parameters: {'learning_rate': 0.0004841251983791936, 'per_device_train_batch_size': 8, 'num_train_epochs': 2}. Best is trial 0 with value: 0.9642857142857143.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-02-01 16:26:39,501] Trial 2 finished with value: 0.9285714285714286 and parameters: {'learning_rate': 0.0003169465104468718, 'per_device_train_batch_size': 16, 'num_train_epochs': 3}. Best is trial 0 with value: 0.9642857142857143.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-02-01 16:41:47,202] Trial 3 finished with value: 0.9642857142857143 and parameters: {'learning_rate': 0.00021073447095197996, 'per_device_train_batch_size': 32, 'num_train_epochs': 5}. Best is trial 0 with value: 0.9642857142857143.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[I 2025-02-01 16:47:52,344] Trial 4 finished with value: 0.8214285714285714 and parameters: {'learning_rate': 3.9687757840299386e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 2}. Best is trial 0 with value: 0.9642857142857143.


In [53]:
print("Best Hyperparameters:", best_trial.hyperparameters)

Best Hyperparameters: {'learning_rate': 0.0002728032165901902, 'per_device_train_batch_size': 16, 'num_train_epochs': 2}


In [54]:
tuning_trainer.model.save_pretrained("my_tuned_model") # not used when pushing model to hub
#tuning_trainer.push_to_hub("my_tuned_model") # to be used when push_to_hub = True
tuning_trainer.processing_class.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

## Inference

In [159]:
my_model = AutoModelForSequenceClassification.from_pretrained("my_tuned_model")
#my_model = AutoModelForSequenceClassification.from_pretrained("lnewbould/my_tuned_model") # to be used when push_to_hub = True
my_tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [161]:
from transformers import pipeline

In [163]:
my_text_classifier = pipeline("text-classification", model=my_model, tokenizer=my_tokenizer)

Device set to use cpu


In [165]:
import random 
random_sample = random.randint(0, 20)
sample = test_data[random_sample]
sample_text = sample['text']
sample_label = sample['label']

In [167]:
my_model_result = my_text_classifier(sample_text)

In [169]:
print("Text to classify:\n", sample_text, 
      "\n\nThe correct label from the dataset:", sample_label, 
      "\nOur model classification:", my_model_result[0]['label'], "with a score of", my_model_result[0]['score'])

Text to classify:
 I was recently watching a video of a police officer who lied to a driver that "a new law passed" that allows police officers to stop someone from recording. The driver was an attorney and knew better.


I was curious about this situation where a law changes that you may not be aware of because it only very recently passes. It seems wrong that someone who may have been not aware it was recently changed would be punished. I imagine it depends on the severity of the law, but is there some sort of grace period due to information not travelling fast enough?


While I think the above scenario is a little extreme, any advice is appreciated.


 

The correct label from the dataset: criminal-law 
Our model classification: criminal-law with a score of 0.2641546428203583


## Pushing model to hub

Will need to move this section up in finished file so hyperparam tuning can be uploaded to hub. 

In [173]:
from huggingface_hub import notebook_login
notebook_login()

In [175]:
my_model.push_to_hub("my_final_model")
my_tokenizer.push_to_hub("my_final_model")

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lnewbould/my_final_model/commit/2bd5ea82b139fd02721ac1dae10c05fc49c59d99', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='2bd5ea82b139fd02721ac1dae10c05fc49c59d99', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lnewbould/my_final_model', endpoint='https://huggingface.co', repo_type='model', repo_id='lnewbould/my_final_model'), pr_revision=None, pr_num=None)

## Gradio

In [133]:
import gradio as gr

In [147]:
demo = gr.Interface.from_pipeline(my_text_classifier)
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://7ce527515988a9431e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
